In [1]:
import open3d as o3d
import numpy as np
import json
# Get cuurent robot position
current_pos = 20
point_cloud = o3d.io.read_point_cloud(f'track_1/{current_pos}.ply')

#visualize the point cloud
# o3d.visualization.draw_geometries([point_cloud], window_name="Point Cloud Viewer")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:


with open(f'track_1/{current_pos}_car_state.json', 'r') as f:
    car_1 = json.load(f)
with open(f'track_1/{current_pos+15}_car_state.json', 'r') as ft:
    car_2 = json.load(ft)

car_pos = np.array([car_2['kinematics_estimated']['position']['x_val'], car_2['kinematics_estimated']['position']['y_val'], car_2['kinematics_estimated']['position']['z_val']])
print(car_pos)
print(car_2)

[6.73133278 1.71448755 0.02601008]
{'speed': 1.151845097541809, 'gear': 1, 'rpm': 499.7661437988281, 'maxrpm': 500.0, 'handbrake': False, 'kinematics_estimated': {'position': {'x_val': 6.731332778930664, 'y_val': 1.7144875526428223, 'z_val': 0.026010077446699142}, 'orientation': {'x_val': 2.454419700370636e-05, 'y_val': -8.552803046768531e-05, 'z_val': 0.27586638927459717, 'w_val': 0.9611960053443909}, 'linear_velocity': {'x_val': 0.9764055609703064, 'y_val': 0.6110478639602661, 'z_val': -2.108308399328962e-05}, 'angular_velocity': {'x_val': 6.667548859695671e-07, 'y_val': -0.0038102702237665653, 'z_val': -0.00044463755330070853}, 'linear_acceleration': {'x_val': -0.014018667861819267, 'y_val': -0.008775300346314907, 'z_val': -0.0018883704906329513}, 'angular_acceleration': {'x_val': 9.44349158089608e-05, 'y_val': -0.2650919556617737, 'z_val': -0.00972615834325552}}, 'timestamp': 1746108922845651500}


In [3]:
# get convert the point cloud to global coordinates
lidar_points = np.asarray(point_cloud.points)
print(lidar_points)
def quaternion_to_rotation_matrix(q):
        qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
        return np.array([
            [1 - 2*qy*qy - 2*qz*qz, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
            [2*qx*qy + 2*qz*qw, 1 - 2*qx*qx - 2*qz*qz, 2*qy*qz - 2*qx*qw],
            [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx*qx - 2*qy*qy]
        ])
def get_vehicle_pose(car_dict):
    vehicle_pose = car_dict['kinematics_estimated']
    print(vehicle_pose)
    pos = vehicle_pose['position']
    orient = vehicle_pose['orientation']
    position_array = np.array([float(pos['x_val']), float(pos['y_val']), float(pos['z_val'])])
    rotation_matrix = quaternion_to_rotation_matrix(orient)
    return position_array, rotation_matrix
def transform_to_world(points, position, rotation_matrix):
    points_rotated = np.dot(points, rotation_matrix.T)
    return points_rotated + position
position_1, rotation_matrix_1 = get_vehicle_pose(car_1)
# get the world coordinates of the points
points_world_1 = transform_to_world(lidar_points, position_1, rotation_matrix_1)
print(points_world_1)
print(car_pos)

[[ 1.56337047e+00  2.73348377e-07 -5.69020331e-01]
 [ 1.56337047e+00  2.73348405e-07 -5.49478948e-01]
 [ 1.68841732e+00  2.95212288e-07 -5.72488487e-01]
 ...
 [ 2.80506873e+00 -1.38890162e-01 -5.83967030e-01]
 [ 2.80506825e+00 -1.38890132e-01 -5.51572382e-01]
 [ 2.80506873e+00 -1.38890147e-01 -5.19318581e-01]]
{'position': {'x_val': 4.105386257171631, 'y_val': -0.25966617465019226, 'z_val': 0.026110583916306496}, 'orientation': {'x_val': 9.580528967489954e-06, 'y_val': -5.2504106861306354e-05, 'z_val': 0.18658044934272766, 'w_val': 0.9824396967887878}, 'linear_velocity': {'x_val': 1.1782411336898804, 'y_val': 0.4667007029056549, 'z_val': -1.2123963642807212e-05}, 'angular_velocity': {'x_val': 4.755701593239792e-05, 'y_val': -0.0015654905000701547, 'z_val': 0.5049094557762146}, 'linear_acceleration': {'x_val': -0.21480172872543335, 'y_val': 0.5163766145706177, 'z_val': -0.0007224251748993993}, 'angular_acceleration': {'x_val': 0.0006045398768037558, 'y_val': -0.09282144904136658, 'z_val

In [4]:
# label the points as 0.5
labels = np.zeros(points_world_1.shape[0]) + 0.5

# then mark the points that are closed to the car as 1.0
distances = np.linalg.norm(points_world_1 - car_pos, axis=1)
print(min(distances), max(distances))
threshold = 1.2

labels[distances < threshold] = 0.0
# create a new point cloud with the labels
point_cloud_with_labels = o3d.geometry.PointCloud()
point_cloud_with_labels.points = o3d.utility.Vector3dVector(lidar_points)
# label the points with the labels that 0.5 as orange and 0.0 as green
colors = np.zeros((len(labels), 3))
colors[labels == 0.5] = [1, 0.5, 0]  # orange
colors[labels == 0.0] = [0, 1, 0]  # green
point_cloud_with_labels.colors = o3d.utility.Vector3dVector(colors)

# visualize the point cloud with labels
# o3d.visualization.draw_geometries([point_cloud_with_labels], window_name="Point Cloud Viewer with Labels")

0.674492208748337 6.957939372657722


In [5]:
# add labels to the point cloud
# point_cloud.colors = o3d.utility.Vector3dVector(np.array([[1, 0, 0]] * len(t)))
# visualize the point cloud with labels
# o3d.visualization.draw_geometries([point_cloud], window_name="Point Cloud Viewer with Labels")

In [6]:
# get the max and min coordinates of the point cloud on y
max_y = point_cloud.get_max_bound()[1]
min_y = point_cloud.get_min_bound()[1]
print("Max Y:", max_y)
print("Min Y:", min_y)
# get the number of points in the point cloud

Max Y: 2.4322943687438965
Min Y: -3.195559024810791


In [7]:
# load the json
import json
with open('track_1/1_car_state.json', 'r') as f:
    data = json.load(f)
data['speed']

0.6853307485580444

In [11]:
import open3d as o3d
import numpy as np
import json

# parameters
current_pos = 50
threshold = 1.2

# 1) load & transform the LiDAR once
pc = o3d.io.read_point_cloud(f'track_1/{current_pos}.ply')
lidar_points = np.asarray(pc.points)

with open(f'track_1/{current_pos}_car_state.json', 'r') as f:
    car_0 = json.load(f)

def quaternion_to_rotation_matrix(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1 - 2*qy*qy - 2*qz*qz,  2*qx*qy - 2*qz*qw,    2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw,      1 - 2*qx*qx - 2*qz*qz,2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw,      2*qy*qz + 2*qx*qw,    1 - 2*qx*qx - 2*qy*qy]
    ])

def get_vehicle_pose(car_dict):
    pos = car_dict['kinematics_estimated']['position']
    ori = car_dict['kinematics_estimated']['orientation']
    position = np.array([pos['x_val'], pos['y_val'], pos['z_val']], dtype=float)
    R = quaternion_to_rotation_matrix(ori)
    return position, R

pos0, R0 = get_vehicle_pose(car_0)
# world‐coords of each LiDAR point
points_world = (R0 @ lidar_points.T).T + pos0

# 2) initialize all labels to 0.5 (orange)
labels = np.full(len(points_world), 0.5, dtype=float)

# 3) loop forward until no points fall within the threshold
i = 1
while True:
    idx = i
    try:
        with open(f'track_1/{idx}_car_state.json', 'r') as f:
            car_i = json.load(f)
        
    except FileNotFoundError:
        print(f"[STOP] no more car_state at frame {idx}")
        break

    pos_i, _ = get_vehicle_pose(car_i)
    dists = np.linalg.norm(points_world - pos_i, axis=1)

    close_mask = dists < threshold
    num_new = np.count_nonzero(close_mask & (labels != 0.0))
    # if num_new == 0:
    #     print(f"[DONE] at frame {idx}, no new points within {threshold} m.")
    #     break

    # **only** mark those still‐unmarked points that are now too close
    labels[close_mask] = 0.0
    print(f"frame {idx}: marked {num_new} additional points as 0.0")

    i += 1

# 4) build a colored point‐cloud once, preserving all 0.0 and 0.5 labels
colors = np.zeros((len(labels), 3))
colors[labels == 0.5] = [1, 0.5, 0]  # orange
colors[labels == 0.0] = [0, 1, 0]    # green

pc_labeled = o3d.geometry.PointCloud()
pc_labeled.points = o3d.utility.Vector3dVector(lidar_points)
pc_labeled.colors = o3d.utility.Vector3dVector(colors)

# 5) visualize
o3d.visualization.draw_geometries(
    [pc_labeled],
    window_name="Sticky‐Labels Point Cloud"
)


frame 1: marked 0 additional points as 0.0
frame 2: marked 0 additional points as 0.0
frame 3: marked 0 additional points as 0.0
frame 4: marked 0 additional points as 0.0
frame 5: marked 0 additional points as 0.0
frame 6: marked 0 additional points as 0.0
frame 7: marked 0 additional points as 0.0
frame 8: marked 0 additional points as 0.0
frame 9: marked 0 additional points as 0.0
frame 10: marked 0 additional points as 0.0
frame 11: marked 0 additional points as 0.0
frame 12: marked 0 additional points as 0.0
frame 13: marked 0 additional points as 0.0
frame 14: marked 0 additional points as 0.0
frame 15: marked 0 additional points as 0.0
frame 16: marked 0 additional points as 0.0
frame 17: marked 0 additional points as 0.0
frame 18: marked 14 additional points as 0.0
frame 19: marked 12 additional points as 0.0
frame 20: marked 28 additional points as 0.0
frame 21: marked 38 additional points as 0.0
frame 22: marked 2 additional points as 0.0
frame 23: marked 4 additional points 

In [23]:
import open3d as o3d
import numpy as np
import json

# params
current_pos = 38
threshold   = 1.2

# 1) load & transform your LiDAR just once
pc = o3d.io.read_point_cloud(f'track_1/{current_pos}.ply')
lidar_pts = np.asarray(pc.points)

with open(f'track_1/{current_pos}_car_state.json','r') as f:
    car0 = json.load(f)

def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz, 2*qx*qy-2*qz*qw,   2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,   1-2*qx*qx-2*qz*qz, 2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,   2*qy*qz+2*qx*qw,   1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    R   = quat2rot(o)
    return pos, R

pos0, R0 = get_pose(car0)
# world-coords of LiDAR
world_pts = (R0 @ lidar_pts.T).T + pos0

# 2) prepare an “ever-close” boolean mask
ever_close = np.zeros(len(world_pts), dtype=bool)

# 3) sweep forward until no frame introduces new “close” points
i = 1
while True:
    idx =  i
    try:
        with open(f'track_1/{idx}_car_state.json','r') as f:
            car_i = json.load(f)
        with open(f'track_1/{idx}_collisioin_info.json','r') as f:
            collision_info = json.load(f)
    except FileNotFoundError:
        print(f"[stop] no more car_state at frame {idx}")
        break

    pos_i, _    = get_pose(car_i)
    dists       = np.linalg.norm(world_pts - pos_i, axis=1)
    close_mask  = dists < threshold

    # record any new close points
    new_hits = close_mask & ~ever_close
    ever_close |= close_mask


    print(f"frame {idx}: +{new_hits.sum()} newly < {threshold} m")
    i += 1

# 4) color the point cloud: gray for “never close”, green for “ever close”
colors = np.ones((len(world_pts), 3)) * 0.7   # default light gray
colors[ever_close] = [0, 1, 0]                # green

pc.colors = o3d.utility.Vector3dVector(colors)

# 5) visualize
o3d.visualization.draw_geometries([pc], window_name="Only‐Zero Labeling")


frame 1: +0 newly < 1.2 m
frame 2: +0 newly < 1.2 m
frame 3: +0 newly < 1.2 m
frame 4: +0 newly < 1.2 m
frame 5: +0 newly < 1.2 m
frame 6: +0 newly < 1.2 m
frame 7: +0 newly < 1.2 m
frame 8: +0 newly < 1.2 m
frame 9: +0 newly < 1.2 m
frame 10: +0 newly < 1.2 m
frame 11: +0 newly < 1.2 m
frame 12: +0 newly < 1.2 m
frame 13: +0 newly < 1.2 m
frame 14: +0 newly < 1.2 m
frame 15: +0 newly < 1.2 m
frame 16: +0 newly < 1.2 m
frame 17: +0 newly < 1.2 m
frame 18: +1 newly < 1.2 m
frame 19: +4 newly < 1.2 m
frame 20: +24 newly < 1.2 m
frame 21: +17 newly < 1.2 m
frame 22: +4 newly < 1.2 m
frame 23: +2 newly < 1.2 m
frame 24: +23 newly < 1.2 m
frame 25: +62 newly < 1.2 m
frame 26: +103 newly < 1.2 m
frame 27: +96 newly < 1.2 m
frame 28: +43 newly < 1.2 m
frame 29: +28 newly < 1.2 m
frame 30: +15 newly < 1.2 m
frame 31: +7 newly < 1.2 m
frame 32: +4 newly < 1.2 m
frame 33: +0 newly < 1.2 m
frame 34: +0 newly < 1.2 m
frame 35: +728 newly < 1.2 m
frame 36: +134 newly < 1.2 m
frame 37: +63 newly < 1

In [ ]:
import open3d as o3d
import numpy as np
import json

# params
current_pos     = 1
threshold       = 1.2

# 1) load & transform LiDAR once
pc             = o3d.io.read_point_cloud(f'track_1/{current_pos}.ply')
lidar_pts      = np.asarray(pc.points)
with open(f'track_1/{current_pos}_car_state.json','r') as f:
    car0        = json.load(f)

def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz, 2*qx*qy-2*qz*qw,   2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,   1-2*qx*qx-2*qz*qz, 2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,   2*qy*qz+2*qx*qw,   1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

pos0, R0      = get_pose(car0)
world_pts     = (R0 @ lidar_pts.T).T + pos0

# 2) two “ever”-masks
ever_close     = np.zeros(len(world_pts), dtype=bool)
ever_collision = np.zeros(len(world_pts), dtype=bool)

# 3) sweep forward through frames
i = 1
while True:
    idx = i
    try:
        with open(f'track_1/{idx}_car_state.json','r') as f:
            car_i = json.load(f)
        with open(f'track_1/{idx}_collision_info.json','r') as f:
            col_i = json.load(f)
    except FileNotFoundError:
        print(f"[stop] no more data at frame {idx}")
        break

    # car‐proximity
    pos_i, _       = get_pose(car_i)
    d_car          = np.linalg.norm(world_pts - pos_i, axis=1)
    close_mask     = d_car < threshold
    new_car_hits   = close_mask & ~ever_close
    ever_close    |= close_mask

    # collision‐proximity
    #  — adapt these keys to your JSON structure —
    cp = col_i['position']
    col_pos = np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
    d_col         = np.linalg.norm(world_pts - col_pos, axis=1)
    col_mask      = d_col < threshold
    new_col_hits  = col_mask & ~ever_collision
    ever_collision |= col_mask

    print(f"frame {idx}: +{new_car_hits.sum()} new car‐hits, +{new_col_hits.sum()} new collision‐hits")
    i += 1

# 4) assemble a single integer label array
#    0 = never-close, 1 = collision, 2 = car-only
labels = np.zeros(len(world_pts), dtype=int)
labels[ever_close]     = 2
labels[ever_collision] = 1

# 5) color map for visualization
colors = np.ones((len(world_pts), 3)) * 0.7   # default gray
colors[labels == 2] = [0, 1, 0]               # green for car
colors[labels == 1] = [1, 0, 0]               # red for collision

pc.colors = o3d.utility.Vector3dVector(colors)

# 6) show
o3d.visualization.draw_geometries([pc], window_name="Car vs Collision Labeling")


frame 1: +0 new car‐hits, +0 new collision‐hits
frame 2: +0 new car‐hits, +0 new collision‐hits
frame 3: +0 new car‐hits, +0 new collision‐hits
frame 4: +0 new car‐hits, +0 new collision‐hits
frame 5: +27 new car‐hits, +0 new collision‐hits
frame 6: +51 new car‐hits, +0 new collision‐hits
frame 7: +41 new car‐hits, +0 new collision‐hits
frame 8: +57 new car‐hits, +0 new collision‐hits
frame 9: +67 new car‐hits, +0 new collision‐hits
frame 10: +46 new car‐hits, +0 new collision‐hits
frame 11: +24 new car‐hits, +0 new collision‐hits
frame 12: +30 new car‐hits, +0 new collision‐hits
frame 13: +15 new car‐hits, +0 new collision‐hits
frame 14: +16 new car‐hits, +0 new collision‐hits
frame 15: +5 new car‐hits, +0 new collision‐hits
frame 16: +0 new car‐hits, +0 new collision‐hits
frame 17: +0 new car‐hits, +0 new collision‐hits
frame 18: +0 new car‐hits, +0 new collision‐hits
frame 19: +0 new car‐hits, +0 new collision‐hits
frame 20: +0 new car‐hits, +0 new collision‐hits
frame 21: +0 new ca

In [11]:
import open3d as o3d
import numpy as np
import json
import os
import time
import imageio

# ─── PARAMETERS ────────────────────────────────────────────────────────────────
start_frame = 0
end_frame   = 93
threshold   = 1.2
output_path = "output.mp4"
fps         = 10
width, height = 800, 600

# ─── HELPERS ──────────────────────────────────────────────────────────────────
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz,   2*qx*qy-2*qz*qw,     2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,     1-2*qx*qx-2*qz*qz,   2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,     2*qy*qz+2*qx*qw,     1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

# ─── SET UP OFFSCREEN VISUALIZER ───────────────────────────────────────────────
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False, width=width, height=height)
pc_vis = o3d.geometry.PointCloud()
vis.add_geometry(pc_vis)
ctr = vis.get_view_control()
ctr.set_front([0.5, -0.5, -1.0])
ctr.set_lookat([0, 0, 0])
ctr.set_up([0, 0, 1])
ctr.set_zoom(0.5)

# ─── SET UP IMAGEIO VIDEO WRITER ───────────────────────────────────────────────
writer = imageio.get_writer(output_path, fps=fps)

# ─── MAIN LOOP ─────────────────────────────────────────────────────────────────
for idx in range(start_frame, end_frame + 1):
    plyf   = f"track_1/{idx}.ply"
    carf   = f"track_1/{idx}_car_state.json"
    collf  = f"track_1/{idx}_collision_info.json"
    if not (os.path.exists(plyf) and os.path.exists(carf) and os.path.exists(collf)):
        print(f"[skip {idx}] missing base files")
        continue

    # load & transform LiDAR into world coords at frame idx
    cloud   = o3d.io.read_point_cloud(plyf)
    pts     = np.asarray(cloud.points)
    car0    = json.load(open(carf))
    pos0, R0 = get_pose(car0)
    world_pts = (R0 @ pts.T).T + pos0

    ever_close     = np.zeros(len(world_pts), dtype=bool)
    ever_collision = np.zeros(len(world_pts), dtype=bool)

    # sweep through future frames to build masks
    j = idx + 1
    while True:
        car_jf  = f"track_1/{j}_car_state.json"
        coll_jf = f"track_1/{j}_collision_info.json"
        if not (os.path.exists(car_jf) and os.path.exists(coll_jf)):
            break

        car_j  = json.load(open(car_jf))
        coll_j = json.load(open(coll_jf))

        # proximity mask
        pos_car, _ = get_pose(car_j)
        d_car      = np.linalg.norm(world_pts - pos_car, axis=1)
        ever_close |= (d_car < threshold)

        # collision mask
        cp        = coll_j['position']
        pos_col   = np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
        d_col     = np.linalg.norm(world_pts - pos_col, axis=1)
        ever_collision |= (d_col < threshold)

        j += 1

    # color code: red=ever collided, green=ever close only, gray otherwise
    colors = np.ones((len(world_pts), 3)) * 0.7
    colors[ever_collision]               = [1, 0, 0]
    colors[ever_close & ~ever_collision] = [0, 1, 0]

    # update visualizer
    pc_vis.points = o3d.utility.Vector3dVector(world_pts)
    pc_vis.colors = o3d.utility.Vector3dVector(colors)
    vis.update_geometry(pc_vis)
    vis.poll_events()
    vis.update_renderer()

    # capture & append to video
    img = np.asarray(vis.capture_screen_float_buffer(do_render=True))   # RGB floats [0,1]
    img_uint8 = (img * 255).astype(np.uint8)                             # uint8 [0,255]
    writer.append_data(img_uint8)                                        # ImageIO expects RGB
    print(f"Added frame {idx} to video.")
    time.sleep(0.1)  # optional pause

# ─── CLEAN UP ─────────────────────────────────────────────────────────────────
writer.close()
vis.destroy_window()
print(f"Video saved to {output_path}")


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
Added frame 0 to video.


OSError: [Errno 22] Invalid argument

FFMPEG COMMAND:
c:\Users\gavin\.conda\envs\lidarAIR\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -y -f rawvideo -vcodec rawvideo -s 800x600 -pix_fmt rgb24 -r 10.00 -i - -an -vcodec libx264 -pix_fmt yuv420p -vf scale=800:608 -v warning c:\Users\gavin\OneDrive\Documents\GitHub\Cosys-AirSim\PythonClient\car\trying\test\output.mp4

FFMPEG STDERR OUTPUT:


In [8]:
import open3d as o3d
import numpy as np
import json
import os
import time

# PARAMETERS
start_frame = 0
end_frame   = 93
threshold   = 1.2

# HELPERS
def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz,   2*qx*qy-2*qz*qw,     2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,     1-2*qx*qx-2*qz*qz,   2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,     2*qy*qz+2*qx*qw,     1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

# VIS SETUP (optional – you can swap for video writer or offscreen)
vis = o3d.visualization.Visualizer()
vis.create_window(visible=False, width=800, height=600)
pc_vis = o3d.geometry.PointCloud()
vis.add_geometry(pc_vis)
ctr = vis.get_view_control()
ctr.set_front([0.5, -0.5, -1.0])
ctr.set_lookat([0,0,0])
ctr.set_up([0,0,1])
ctr.set_zoom(0.5)

# DOUBLE–LOOP
for idx in range(start_frame, end_frame + 1):
    plyf = f"track_1/{idx}.ply"
    carf = f"track_1/{idx}_car_state.json"
    collf= f"track_1/{idx}_collision_info.json"
    if not (os.path.exists(plyf) and os.path.exists(carf) and os.path.exists(collf)):
        print(f"[skip {idx}] missing base files")
        continue

    # --- load & transform LiDAR at base frame idx
    cloud = o3d.io.read_point_cloud(plyf)
    pts   = np.asarray(cloud.points)
    car0  = json.load(open(carf))
    pos0, R0 = get_pose(car0)
    world_pts = (R0 @ pts.T).T + pos0

    # --- init “ever” masks
    ever_close     = np.zeros(len(world_pts), dtype=bool)
    ever_collision = np.zeros(len(world_pts), dtype=bool)

    # --- sweep through future frames j = idx+1, idx+2, ...
    j = idx + 1
    while True:
        car_jf  = f"track_1/{j}_car_state.json"
        coll_jf = f"track_1/{j}_collision_info.json"
        if not (os.path.exists(car_jf) and os.path.exists(coll_jf)):
            print(f"  [stop inner @{j}] no more future data")
            break

        # load future car state & collision
        car_j = json.load(open(car_jf))
        coll_j = json.load(open(coll_jf))

        # update car‐proximity mask
        pos_car, _ = get_pose(car_j)
        d_car      = np.linalg.norm(world_pts - pos_car, axis=1)
        ever_close |= (d_car < threshold)

        # update collision mask
        cp     = coll_j['position']
        pos_col= np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
        d_col  = np.linalg.norm(world_pts - pos_col, axis=1)
        ever_collision |= (d_col < threshold)

        j += 1

    # --- after inner loop: color & visualize
    colors = np.ones((len(world_pts), 3)) * 0.7
    colors[ever_collision]                    = [1, 0, 0]  # red
    colors[ever_close & ~ever_collision]      = [0, 1, 0]  # green

    pc_vis.points = o3d.utility.Vector3dVector(world_pts)
    pc_vis.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries(
        [pc_vis],
        window_name = f"Base frame {idx}",
        width       = 800,
        height      = 600
    )
    time.sleep(0.1)
    


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
  [stop inner @95] no more future data
  [stop inner @95] no more future data
  [stop inner @95] no more future data


KeyboardInterrupt: 

In [ ]:
import open3d as o3d
import numpy as np
import json
import os
import time
import imageio
import cv2
# PARAMETERS
start_frame = 0
end_frame   = 93
threshold   = 1.2
output_path = "output1.mp4"
fps         = 10
width, height = 800, 600

def quat2rot(q):
    qw, qx, qy, qz = q['w_val'], q['x_val'], q['y_val'], q['z_val']
    return np.array([
        [1-2*qy*qy-2*qz*qz,   2*qx*qy-2*qz*qw,     2*qx*qz+2*qy*qw],
        [2*qx*qy+2*qz*qw,     1-2*qx*qx-2*qz*qz,   2*qy*qz-2*qx*qw],
        [2*qx*qz-2*qy*qw,     2*qy*qz+2*qx*qw,     1-2*qx*qx-2*qy*qy]
    ])

def get_pose(state):
    p = state['kinematics_estimated']['position']
    o = state['kinematics_estimated']['orientation']
    pos = np.array([p['x_val'], p['y_val'], p['z_val']], dtype=float)
    return pos, quat2rot(o)

# OFF-SCREEN OPEN3D SETUP
vis = o3d.visualization.Visualizer()
vis.create_window(visible=True, width=width, height=height)
pc_vis = o3d.geometry.PointCloud()
vis.add_geometry(pc_vis)
# ctr = vis.get_view_control()
# ctr.set_front([0.5, 10, -1.0])
# ctr.set_lookat([0, 0, 0])
# ctr.set_up([0, 0, 1])
# ctr.set_zoom(0.5)

# # ─── ImageIO writer (note the macro_block_size tweak) ───────────────────────────
# writer = imageio.get_writer(
#     output_path,
#     fps=fps,
#     codec="libx264",
#     macro_block_size=1   # <— this disables the automatic 16-pixel padding
# )

for idx in range(start_frame, end_frame + 1):
    plyf   = f"track_1/{idx}.ply"
    carf   = f"track_1/{idx}_car_state.json"
    collf  = f"track_1/{idx}_collision_info.json"
    if not (os.path.exists(plyf) and os.path.exists(carf) and os.path.exists(collf)):
        print(f"[skip {idx}] missing base files")
        continue

    cloud = o3d.io.read_point_cloud(plyf)
    pts   = np.asarray(cloud.points)
    car0  = json.load(open(carf))
    pos0, R0 = get_pose(car0)
    world_pts = (R0 @ pts.T).T + pos0

    ever_close     = np.zeros(len(world_pts), dtype=bool)
    ever_collision = np.zeros(len(world_pts), dtype=bool)

    j =  1
    while True:
        car_jf  = f"track_1/{j}_car_state.json"
        coll_jf = f"track_1/{j}_collision_info.json"
        if not (os.path.exists(car_jf) and os.path.exists(coll_jf)):
            break

        car_j  = json.load(open(car_jf))
        coll_j = json.load(open(coll_jf))

        pos_car, _ = get_pose(car_j)
        d_car      = np.linalg.norm(world_pts - pos_car, axis=1)
        ever_close |= (d_car < (threshold-0.3))
        if coll_j['has_collided']:
            cp      = coll_j['position']
            pos_col = np.array([cp['x_val'], cp['y_val'], cp['z_val']], dtype=float)
            d_col   = np.linalg.norm(world_pts - pos_col, axis=1)
            ever_collision |= (d_col < (threshold+0.3))

        j += 1

    colors = np.ones((len(world_pts), 3)) * 0.7
    colors[ever_collision]               = [1, 0, 0]
    colors[ever_close & ~ever_collision] = [0, 1, 0]

    pc_vis.points = o3d.utility.Vector3dVector(world_pts)
    pc_vis.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries(
        [pc_vis],
        window_name = f"Base frame {idx}",
        width       = 800,
        height      = 600
    )
    # capture image from visualizer
    # img = np.asarray(vis.capture_screen_float_buffer(do_render=True))
    # img = np.asarray(vis.capture_screen_float_buffer(do_render=True))
    # cv2.imshow("img", img)
    # img_uint8 = (img * 255).astype(np.uint8)
    # writer.append_data(img_uint8)
    # print(f"Added frame {idx} to video.")
    time.sleep(0.1)

# writer.close()
# vis.destroy_window()
# print(f"Video saved to {output_path}")


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
